# Notebook for plotting AuxTel gains.

Initially written 29 Sep 2022 by Craig Lage.

In [ ]:
import sys, os, glob, time
import numpy as np
import astropy.io.fits as pf
import matplotlib.pyplot as plt
from lsst.daf.butler import Butler

In [ ]:
butler1 = Butler("/repo/embargo", collections=["u/cslage/sdf/latiss/ptc_20230321_v26"])
exposure2 = 2023032100093
butler2 = Butler("/repo/embargo", collections=["u/cslage/sdf/latiss/ptc_20230321_v26_noRG"])
exposure1 = 2023032100213

In [ ]:
butlers = [butler1, butler2]
exposures = [exposure1, exposure2]
run_names = ['20230321-V26', '20230321-V26-noRG']
markers = ['o', 'x']

In [ ]:
plt.figure(figsize=(8,8))
plt.suptitle("AuxTel Gains", fontsize=24)
for i,butler in enumerate(butlers):
    run_name = run_names[i]
    plt.subplot(1,1,1)
    ptcDataset = butler.get('ptc', exposure=exposures[i], detector=0)
    gain_data = ptcDataset.gain
    gain_err_data = ptcDataset.gainErr
    amps = gain_data.keys()
    gains = []
    gain_err = []
    amp_nums = []
    for ii, amp in enumerate(amps):
        gains.append(gain_data[amp])
        gain_err.append(gain_err_data[amp])
        amp_nums.append(ii)
    plt.errorbar(amp_nums, gains, yerr=gain_err, marker = markers[i], label = run_name)
    plt.ylim(0.5, 1.5)
    plt.ylabel("Gain", fontsize = 12)
    plt.xticks(amp_nums,amps, fontsize=8)
    plt.legend(loc = 'upper right', fontsize = 12)
plt.savefig("/sdf/group/rubin/u/cslage/AuxTel/gains/Gain_Comparison_21Mar23.pdf")


In [ ]:
plt.figure(figsize=(8,8))
plt.suptitle("AuxTel Noise", fontsize=24)
for i,butler in enumerate(butlers):
    run_name = run_names[i]
    plt.subplot(1,1,1)
    ptcDataset = butler.get('ptc', exposure=exposures[i], detector=0)
    noise_data = ptcDataset.noise
    noise_err_data = ptcDataset.noiseErr
    amps = noise_data.keys()
    noises = []
    noise_err = []
    amp_nums = []
    for ii, amp in enumerate(amps):
        noises.append(noise_data[amp])
        noise_err.append(noise_err_data[amp])
        amp_nums.append(ii)
    plt.errorbar(amp_nums, noises, yerr=noise_err, marker = markers[i], label = run_name)
    #plt.ylim(0.5, 1.5)
    plt.ylabel("Noise", fontsize = 12)
    plt.xticks(amp_nums,amps, fontsize=8)
    plt.legend(loc = 'upper right', fontsize = 12)
plt.savefig("/sdf/group/rubin/u/cslage/AuxTel/gains/Noise_Comparison_17Mar23.pdf")


In [ ]:
plt.figure(figsize=(8,8))
plt.suptitle("AuxTel Noise", fontsize=24)
for i,butler in enumerate(butlers):
    run_name = run_names[i]
    plt.subplot(1,1,1)
    cpPtcExtract = butler.get('cpPtcExtract', exposure=exposures[i], detector=0)
    noise_data = cpPtcExtract.noise
    noise_err_data = cpPtcExtract.noiseErr
    ptcDataset = butler.get('ptc', exposure=exposures[i], detector=0)
    gain_data = ptcDataset.gain

    amps = gain_data.keys()
    noises = []
    noise_err = []
    amp_nums = []
    for ii, amp in enumerate(amps):
        noises.append(noise_data[amp] * gain_data[amp])
        noise_err.append(noise_err_data[amp]* gain_data[amp])
        amp_nums.append(ii)
    plt.errorbar(amp_nums, noises, yerr=noise_err, marker = markers[i], label = run_name)
    #plt.ylim(0.5, 1.5)
    plt.ylabel("Overscan Noise in e-", fontsize = 12)
    plt.xticks(amp_nums,amps, fontsize=8)
    plt.legend(loc = 'upper right', fontsize = 12)
plt.savefig("/sdf/group/rubin/u/cslage/AuxTel/gains/Noise_Overscan_Comparison_17Mar23.pdf")


In [ ]:
plt.figure(figsize=(8,8))
plt.suptitle("Impact of No RG - 21Mar23", fontsize=24)
plt.subplots_adjust(hspace = 0.3)
for i,butler in enumerate(butlers):
    run_name = run_names[i]
    ptcDataset = butler.get('ptc', exposure=exposures[i], detector=0)
    gain_data = ptcDataset.gain
    gain_err_data = ptcDataset.gainErr
    noise_data = ptcDataset.noise
    noise_err_data = ptcDataset.noiseErr
    fitPars = ptcDataset.ptcFitPars
    fitPars_err = ptcDataset.ptcFitParsError

    amps = gain_data.keys()
    gains = []
    gain_err = []
    noises = []
    noise_err = []
    a00s = []
    a00s_err = []
    amp_nums = []
    for ii, amp in enumerate(amps):
        gains.append(gain_data[amp])
        gain_err.append(gain_err_data[amp])
        noises.append(noise_data[amp] * gain_data[amp])
        noise_err.append(noise_err_data[amp] * gain_data[amp])
        a00s.append(-fitPars[amp][0])
        a00s_err.append(fitPars_err[amp][0])
        amp_nums.append(ii)
    plt.subplot(3,1,1)
    plt.title("Gains")
    plt.errorbar(amp_nums, gains, yerr=gain_err, marker = markers[i], label = run_name)
    plt.ylim(1.0, 1.5)
    plt.ylabel("Gain", fontsize = 12)
    plt.xticks(amp_nums,amps, fontsize=8)
    plt.legend(loc = 'lower left', fontsize = 12)
    plt.subplot(3,1,2)
    plt.title("Noise")
    plt.errorbar(amp_nums, noises, yerr=noise_err, marker = markers[i], label = run_name)
    #plt.ylim(1.0, 1.5)
    plt.ylabel("Noise(e-)", fontsize = 12)
    plt.xticks(amp_nums,amps, fontsize=8)
    plt.legend(loc = 'lower left', fontsize = 12)
    plt.subplot(3,1,3)
    plt.title("A00")
    plt.errorbar(amp_nums, a00s, yerr=a00s_err, marker = markers[i], label = run_name)
    #plt.ylim(1.0, 1.5)
    plt.ylabel("-A00", fontsize = 12)
    plt.xticks(amp_nums,amps, fontsize=8)
    plt.legend(loc = 'lower left', fontsize = 12)
plt.savefig("/sdf/group/rubin/u/cslage/AuxTel/gains/Parameter_Comparison_21Mar23.pdf")
